In [1]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize


# Testando GA Simples do Pymoo

In [2]:
problem = get_problem("g1")

algorithm = GA(
    pop_size=100,
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [1.         1.         1.         1.         1.         1.
 1.         1.         1.         2.99551991 2.99431868 2.99999328
 1.        ]
F = [-14.98983187]
